<a href="https://colab.research.google.com/github/redinbluesky/nlp-with-transformers/blob/main/05-텍스트_생성.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  목차
* [Chapter 0 개요](#chapter0)
* [Chapter 1 일관성 있는 텍스트 생성의 어려움](#chapter1)
* [Chapter 2 그리디 서치 디코딩](#chapter2)

## Chapter 0 개요 <a class="anchor" id="chapter0"></a>
1. 트랜스포머 기반 언어 모델은 사람이 작성한 텍스트와 거의 구분되지 않는 텍스트를 생성한다.

2. 아래의 그림은 언어모델이 사전 훈련하는 동안 덧셈, 단어 철자 배열, 번역 같은 문맥 기반으로 다음 토큰을 예측하는 작업 시퀸스에 어떻게 노출되는지 보연준다.

   ![언어모델 훈련](image/05_00_language_modeling_tasks.png) 

## Chapter 1 일관성 있는 텍스트 생성의 어려움 <a class="anchor" id="chapter1"></a>
1. 모델의 확률 출력을 텍스트로 변환하려면 디코딩 방법이 필요한데, 텍스트 생성만 따르는 특수한 어려움이 있다.
    - 디코딩은 반복적으로 수행되므로 입력이 모델의 정방향 패스를 한 번 통과할 때보다 많은 계산이 필요하다.
    - 생성된 텍스트의 품직화 다양성은 디코딩 방법과 이에 관련되 하이퍼파리미터에 따라 달라진다.

2. GPT-2 코델은 문맥 시퀸스 x=x₁, x₂, ..., xₙ이 주어졌을 때 다음 토큰 xₙ₊₁의 확률 분포 P(xₙ₊₁|x)를 예측한다. 
    - 충분한 훈련 데이터를 획득하기란 어렵기 때문에, 일반적으호 확률 연쇄 법칙을 사용해 조건부 확률의 곱으로 전체 시퀸스의 확률을 계산한다.
    - P(x)=∏(i=1 to n) P(xᵢ|x₁, x₂, ..., xᵢ₋₁)

3. 조건부 확률로 자기회귀 언어 모델링은 문장의 이전 단어가 주어지면 다음 단어를 예측한다는 직관을 얻을 수 있다.
    - 이런 사전 훈련 목표는 과거와 미래의 문맥을 모두 사용해 마스킹된 토큰을 예측하는 BERT와 대조적이다.

4. 다음 토큰 예측 작업이 임의의 길이를 가진 텍스트 시퀸스를 생성할 대 어떻게 적용할지 예상해보자
    - "Transfomers are the"같은 프롬프트로 시작하면 모델은 다음  토큰을 예측한다.
    - 다음 토큰이 결저오디면 이를 프롬프트에 추가해 새로운 입력 시퀸스를 만들고 또 다른 토큰을 생성한다.
    - 이 과정을 원하는 길이나 종료 토큰을 만나기 전까지 텍스트 생성을 반복한다.
    - 출력 시퀸스가 입력 프롬프트에 따라 결정되므로 이런 종류의 텍스트 생성을 종종 조건부 텍스트 생성이라고 한다.
    
        ![텍스트 생성](image/05_01_text_generation.png)

5. 이 과정의 핵심은 각 타임스텝에서 어떤 토큰을 선택할지 결정하는 디코딩 방법에 있다.
    - 언어 모델의 해드는 각 스텝에서 어휘사전에 있는 토큰마다 로짓  zᵢ를 출력한다.
    - 그런 다음 소프트맥스 함수를 적용해 다음 토큰 wᵢ의 확률 분포를 계산한다.
        - P(yⱼ=wᵢ|y<ⱼ, x) = softmax(zⱼᵢ)
    - 대부분의 코딩 방법으 다음과 같은 ŷ을 선택해 전체적으로 확률이 가정 높은 시퀸스를 찾는다.
        - ŷ = argmax P(y|x) = argmax ∏(j=1 to m) P(yⱼ|y<ⱼ, x)
    - 직접 ŷ를 찾으려면 언어 모델로 가증한 모든 시퀸스를 평가해야 하므로 계산적으로 불가능하다.
    - 대신 근사 알고리즘을 사용해 효율적으로 높은 확률의 시퀸스를 찾는다.

## Chapter 2 그리디 서치 디코딩 <a class="anchor" id="chapter2"></a>